<font face="微软雅黑" color=green size=5>Julia机器学习Ⅰ</font>

# 引言

In [1]:
VERSION

v"1.7.1"

首先，使用using预加载一些包，以避免稍后的延迟。

In [15]:
using InlineStrings  
using WeakRefStrings  
using ScikitLearn  
using CSV  
using MLJScikitLearnInterface  
using CorpusLoaders  
using ScientificTypes
using OpenML
using MLJBase
using MLJEnsembles
using MLJIteration
using MLJTuning
using MLJSerialization
using MLJModels
using MLJ

# 数据表示

## “科学”类型

为了帮助您关注数据的预期用途或解释，MLJ模型使用“科学”类型而不是机器类型指定数据要求。“科学”类型是普通的Julia类型（例如，它可以用于方法调用），但它通常没有实例。其他基本的“科学”类型包括：Missing（缺失值）、Textual（文本）、OrderedFactor（有序因子）、Multiclass（多类）、Continuous（连续值）、Count（计数值）

In [3]:
scitype(3.141) #X的“科学”类型，与现行公约规定的机器类型不同。

Continuous

In [4]:
time = [2.3, 4.5, 4.2, 1.8, 7.1]
scitype(time)

AbstractVector{Continuous} (alias for AbstractArray{Continuous, 1})

In [5]:
height = [185, 153, 163, 114, 180]
scitype(height)

AbstractVector{Count} (alias for AbstractArray{Count, 1})

In [6]:
height = coerce(height, Continuous) #返回“科学”类型为S的数组A的新版本。

5-element Vector{Float64}:
 185.0
 153.0
 163.0
 114.0
 180.0

In [7]:
exam_mark = ["rotten", "great", "bla",  missing, "great"]
scitype(exam_mark)

AbstractVector{Union{Missing, Textual}} (alias for AbstractArray{Union{Missing, Textual}, 1})

In [8]:
exam_mark = coerce(exam_mark, OrderedFactor)

┌ Info: Trying to coerce from `Union{Missing, String}` to `OrderedFactor`.
│ Coerced to `Union{Missing,OrderedFactor}` instead.
└ @ ScientificTypes D:\TongYuan\.julia\packages\ScientificTypes\3irls\src\convention\coerce.jl:174


5-element CategoricalArrays.CategoricalArray{Union{Missing, String},1,UInt32}:
 "rotten"
 "great"
 "bla"
 missing
 "great"

In [9]:
levels(exam_mark) #返回集合x中出现或可能出现的唯一值的向量。除非传递skipmissing=false，否则将跳过缺少的值。

3-element Vector{String}:
 "bla"
 "great"
 "rotten"

## 二维数据

MLJ模型通常将二维数据表示为表格形式，表的最简单示例可能是julia列表，它只是等长向量的命名元组。

In [10]:
column_table = (h=height, e=exam_mark, t=time)

(h = [185.0, 153.0, 163.0, 114.0, 180.0],
 e = Union{Missing, CategoricalArrays.CategoricalValue{String, UInt32}}["rotten", "great", "bla", missing, "great"],
 t = [2.3, 4.5, 4.2, 1.8, 7.1],)

In [11]:
scitype(column_table)

Table{Union{AbstractVector{Union{Missing, OrderedFactor{3}}}, AbstractVector{Continuous}}}

In [12]:
schema(column_table) #检查表格对象的列类型和类型。如果无法检查列类型和/或scitypes，则返回nothing。

┌───────┬──────────────────────────────────┬────────────────────────────────────
│ names │ scitypes                         │ types                             ⋯
├───────┼──────────────────────────────────┼────────────────────────────────────
│ h     │ Continuous                       │ Float64                           ⋯
│ e     │ Union{Missing, OrderedFactor{3}} │ Union{Missing, CategoricalValue{S ⋯
│ t     │ Continuous                       │ Float64                           ⋯
└───────┴──────────────────────────────────┴────────────────────────────────────
                                                                1 column omitted


In [13]:
dict_table = Dict(:h => height, :e => exam_mark, :t => time)
schema(dict_table)

┌───────┬──────────────────────────────────┬────────────────────────────────────
│ names │ scitypes                         │ types                             ⋯
├───────┼──────────────────────────────────┼────────────────────────────────────
│ e     │ Union{Missing, OrderedFactor{3}} │ Union{Missing, CategoricalValue{S ⋯
│ h     │ Continuous                       │ Float64                           ⋯
│ t     │ Continuous                       │ Float64                           ⋯
└───────┴──────────────────────────────────┴────────────────────────────────────
                                                                1 column omitted


In [14]:
row_table = [(a=1, b=3.4),
             (a=2, b=4.5),
             (a=3, b=5.6)]
schema(row_table)

┌───────┬────────────┬─────────┐
│ names │ scitypes   │ types   │
├───────┼────────────┼─────────┤
│ a     │ Count      │ Int64   │
│ b     │ Continuous │ Float64 │
└───────┴────────────┴─────────┘


In [15]:
import DataFrames
df = DataFrames.DataFrame(column_table)

,h,e,t
,Float64,Cat…?,Float64
1,185.0,rotten,2.3
2,153.0,great,4.5
3,163.0,bla,4.2
4,114.0,missing,1.8
5,180.0,great,7.1


In [16]:
schema(df) == schema(column_table)

true

In [18]:
using CSV
file = CSV.File("D:\\Jupyter notebook\\DataSet\\julia\\horse.csv")
schema(file) 

┌─────────────────────────┬────────────┬─────────┐
│ names                   │ scitypes   │ types   │
├─────────────────────────┼────────────┼─────────┤
│ surgery                 │ Count      │ Int64   │
│ age                     │ Count      │ Int64   │
│ rectal_temperature      │ Continuous │ Float64 │
│ pulse                   │ Count      │ Int64   │
│ respiratory_rate        │ Count      │ Int64   │
│ temperature_extremities │ Count      │ Int64   │
│ mucous_membranes        │ Count      │ Int64   │
│ capillary_refill_time   │ Count      │ Int64   │
│ pain                    │ Count      │ Int64   │
│ peristalsis             │ Count      │ Int64   │
│ abdominal_distension    │ Count      │ Int64   │
│ packed_cell_volume      │ Continuous │ Float64 │
│ total_protein           │ Continuous │ Float64 │
│ outcome                 │ Count      │ Int64   │
│ surgical_lesion         │ Count      │ Int64   │
│ cp_data                 │ Count      │ Int64   │
└─────────────────────────┴────

大多数MLJ模型不接受矩阵代替表格，但您可以将矩阵封装为表格。

In [19]:
matrix_table = MLJ.table(rand(2,3))
schema(matrix_table)

┌───────┬────────────┬─────────┐
│ names │ scitypes   │ types   │
├───────┼────────────┼─────────┤
│ x1    │ Continuous │ Float64 │
│ x2    │ Continuous │ Float64 │
│ x3    │ Continuous │ Float64 │
└───────┴────────────┴─────────┘


## 确定表格数据中的“科学”类型

为了说明我们如何更正表格中的“科学”数据类型，我们利用了Horse的数据集。

In [20]:
using CSV
file = CSV.File("D:\\Jupyter notebook\\DataSet\\julia\\horse.csv");
horse = DataFrames.DataFrame(file); # convert to data frame without copying columns
first(horse, 4)

,surgery,age,rectal_temperature,pulse,respiratory_rate,temperature_extremities
,Int64,Int64,Float64,Int64,Int64,Int64
1,2,1,38.5,66,66,3
2,1,1,39.2,88,88,3
3,2,1,38.3,40,40,1
4,1,9,39.1,164,164,4


从 UCI 文档中，我们可以推测应该如何解释每个变量（我们工作流程中的一个步骤，不能可靠地留给计算机）：

variable                    | scientific type (interpretation)
----------------------------|-----------------------------------
`:surgery`                  | Multiclass
`:age`                      | Multiclass
`:rectal_temperature`       | Continuous
`:pulse`                    | Continuous
`:respiratory_rate`         | Continuous
`:temperature_extremities`  | OrderedFactor
`:mucous_membranes`         | Multiclass
`:capillary_refill_time`    | Multiclass
`:pain`                     | OrderedFactor
`:peristalsis`              | OrderedFactor
`:abdominal_distension`     | OrderedFactor
`:packed_cell_volume`       | Continuous
`:total_protein`            | Continuous
`:outcome`                  | Multiclass
`:surgical_lesion`          | OrderedFactor
`:cp_data`                  | Multiclass

让我们看看 MLJ 将如何实际解释数据，因为它是当前编码的：

In [21]:
schema(horse)

┌─────────────────────────┬────────────┬─────────┐
│ names                   │ scitypes   │ types   │
├─────────────────────────┼────────────┼─────────┤
│ surgery                 │ Count      │ Int64   │
│ age                     │ Count      │ Int64   │
│ rectal_temperature      │ Continuous │ Float64 │
│ pulse                   │ Count      │ Int64   │
│ respiratory_rate        │ Count      │ Int64   │
│ temperature_extremities │ Count      │ Int64   │
│ mucous_membranes        │ Count      │ Int64   │
│ capillary_refill_time   │ Count      │ Int64   │
│ pain                    │ Count      │ Int64   │
│ peristalsis             │ Count      │ Int64   │
│ abdominal_distension    │ Count      │ Int64   │
│ packed_cell_volume      │ Continuous │ Float64 │
│ total_protein           │ Continuous │ Float64 │
│ outcome                 │ Count      │ Int64   │
│ surgical_lesion         │ Count      │ Int64   │
│ cp_data                 │ Count      │ Int64   │
└─────────────────────────┴────

In [22]:
autotype(horse)

Dict{Symbol, Type} with 11 entries:
  :abdominal_distension    => OrderedFactor
  :pain                    => OrderedFactor
  :surgery                 => OrderedFactor
  :mucous_membranes        => OrderedFactor
  :surgical_lesion         => OrderedFactor
  :outcome                 => OrderedFactor
  :capillary_refill_time   => OrderedFactor
  :age                     => OrderedFactor
  :temperature_extremities => OrderedFactor
  :peristalsis             => OrderedFactor
  :cp_data                 => OrderedFactor

好的，这并不完美，但朝着正确的方向迈出了一步，我们这样实现：

In [23]:
coerce!(horse, autotype(horse));
schema(horse)

┌─────────────────────────┬──────────────────┬──────────────────────────────────
│ names                   │ scitypes         │ types                           ⋯
├─────────────────────────┼──────────────────┼──────────────────────────────────
│ surgery                 │ OrderedFactor{2} │ CategoricalValue{Int64, UInt32} ⋯
│ age                     │ OrderedFactor{2} │ CategoricalValue{Int64, UInt32} ⋯
│ rectal_temperature      │ Continuous       │ Float64                         ⋯
│ pulse                   │ Count            │ Int64                           ⋯
│ respiratory_rate        │ Count            │ Int64                           ⋯
│ temperature_extremities │ OrderedFactor{4} │ CategoricalValue{Int64, UInt32} ⋯
│ mucous_membranes        │ OrderedFactor{6} │ CategoricalValue{Int64, UInt32} ⋯
│ capillary_refill_time   │ OrderedFactor{3} │ CategoricalValue{Int64, UInt32} ⋯
│ pain                    │ OrderedFactor{5} │ CategoricalValue{Int64, UInt32} ⋯
│ peristalsis             │ 

所有剩余的计数数据应该是连续的：

In [24]:
coerce!(horse, Count => Continuous);
schema(horse)

┌─────────────────────────┬──────────────────┬──────────────────────────────────
│ names                   │ scitypes         │ types                           ⋯
├─────────────────────────┼──────────────────┼──────────────────────────────────
│ surgery                 │ OrderedFactor{2} │ CategoricalValue{Int64, UInt32} ⋯
│ age                     │ OrderedFactor{2} │ CategoricalValue{Int64, UInt32} ⋯
│ rectal_temperature      │ Continuous       │ Float64                         ⋯
│ pulse                   │ Continuous       │ Float64                         ⋯
│ respiratory_rate        │ Continuous       │ Float64                         ⋯
│ temperature_extremities │ OrderedFactor{4} │ CategoricalValue{Int64, UInt32} ⋯
│ mucous_membranes        │ OrderedFactor{6} │ CategoricalValue{Int64, UInt32} ⋯
│ capillary_refill_time   │ OrderedFactor{3} │ CategoricalValue{Int64, UInt32} ⋯
│ pain                    │ OrderedFactor{5} │ CategoricalValue{Int64, UInt32} ⋯
│ peristalsis             │ 

In [25]:
coerce!(horse,
        :surgery               => Multiclass,
        :age                   => Multiclass,
        :mucous_membranes      => Multiclass,
        :capillary_refill_time => Multiclass,
        :outcome               => Multiclass,
        :cp_data               => Multiclass);
schema(horse)

┌─────────────────────────┬──────────────────┬──────────────────────────────────
│ names                   │ scitypes         │ types                           ⋯
├─────────────────────────┼──────────────────┼──────────────────────────────────
│ surgery                 │ Multiclass{2}    │ CategoricalValue{Int64, UInt32} ⋯
│ age                     │ Multiclass{2}    │ CategoricalValue{Int64, UInt32} ⋯
│ rectal_temperature      │ Continuous       │ Float64                         ⋯
│ pulse                   │ Continuous       │ Float64                         ⋯
│ respiratory_rate        │ Continuous       │ Float64                         ⋯
│ temperature_extremities │ OrderedFactor{4} │ CategoricalValue{Int64, UInt32} ⋯
│ mucous_membranes        │ Multiclass{6}    │ CategoricalValue{Int64, UInt32} ⋯
│ capillary_refill_time   │ Multiclass{3}    │ CategoricalValue{Int64, UInt32} ⋯
│ pain                    │ OrderedFactor{5} │ CategoricalValue{Int64, UInt32} ⋯
│ peristalsis             │ 

## 第1部分资源

- From the MLJ manual:
   - [A preview of data type specification in
  MLJ](https://alan-turing-institute.github.io/MLJ.jl/dev/getting_started/#A-preview-of-data-type-specification-in-MLJ-1)
   - [Data containers and scientific types](https://alan-turing-institute.github.io/MLJ.jl/dev/getting_started/#Data-containers-and-scientific-types-1)
   - [Working with Categorical Data](https://alan-turing-institute.github.io/MLJ.jl/dev/working_with_categorical_data/)
- [Summary](https://juliaai.github.io/ScientificTypes.jl/dev/#Summary-of-the-default-convention) of the MLJ convention for representing scientific types
- [ScientificTypes.jl](https://juliaai.github.io/ScientificTypes.jl/dev/)
- From Data Science Tutorials:
    - [Data interpretation: Scientific Types](https://juliaai.github.io/DataScienceTutorials.jl/data/scitype/)
    - [Horse colic data](https://juliaai.github.io/DataScienceTutorials.jl/end-to-end/horse/)
- [UCI Horse Colic Data Set](http://archive.ics.uci.edu/ml/datasets/Horse+Colic)

# 选择、训练和评估模型

这次，我们将从OpenML获取鸢尾花数据：

In [16]:
using RDatasets

In [16]:
iris = dataset("datasets","iris")

,SepalLength,SepalWidth,PetalLength,PetalWidth,Species
,Float64,Float64,Float64,Float64,Cat…
1,5.1,3.5,1.4,0.2,setosa
2,4.9,3.0,1.4,0.2,setosa
3,4.7,3.2,1.3,0.2,setosa
4,4.6,3.1,1.5,0.2,setosa
5,5.0,3.6,1.4,0.2,setosa
6,5.4,3.9,1.7,0.4,setosa
7,4.6,3.4,1.4,0.3,setosa
8,5.0,3.4,1.5,0.2,setosa
9,4.4,2.9,1.4,0.2,setosa


In [23]:
dataset("datasets","hospital")

LoadError: Unable to locate dataset file D:\TongYuan\.julia\packages\RDatasets\fNG6F\src\..\data\datasets\hospital.rda or D:\TongYuan\.julia\packages\RDatasets\fNG6F\src\..\data\datasets\hospital.csv.gz

In [22]:
OpenML.list_datasets()[:name]

4319-element Vector{String}:
 "anneal"
 "kr-vs-kp"
 "labor"
 "arrhythmia"
 "letter"
 "audiology"
 "liver-disorders"
 "autos"
 "lymph"
 "balance-scale"
 "mfeat-factors"
 "breast-cancer"
 "mfeat-fourier"
 ⋮
 "Iris"
 "Brazilian_houses_reproduced"
 "vehicle_reproduced"
 "iris_reproduced"
 "credit-approval_reproduced"
 "electricity"
 "eye_movements"
 "KDDCup09_upselling"
 "covertype"
 "rl"
 "road-safety"
 "compass"

In [18]:
OpenML.describe_dataset(61)

**Author**: R.A. Fisher   **Source**: [UCI](https://archive.ics.uci.edu/ml/datasets/Iris) - 1936 - Donated by Michael Marshall   **Please cite**:   

**Iris Plants Database**   This is perhaps the best known database to be found in the pattern recognition literature.  Fisher's paper is a classic in the field and is referenced frequently to this day.  (See Duda & Hart, for example.)  The data set contains 3 classes of 50 instances each, where each class refers to a type of iris plant.  One class is     linearly separable from the other 2; the latter are NOT linearly separable from each other.

Predicted attribute: class of iris plant.   This is an exceedingly simple domain.  

### Attribute Information:

```
1. sepal length in cm
2. sepal width in cm
3. petal length in cm
4. petal width in cm
5. class: 
   -- Iris Setosa
   -- Iris Versicolour
   -- Iris Virginica
```


In [18]:
#iris = OpenML.load(61); # a row table
iris = DataFrames.DataFrame(iris);
first(iris, 4)

,SepalLength,SepalWidth,PetalLength,PetalWidth,Species
,Float64,Float64,Float64,Float64,Cat…
1,5.1,3.5,1.4,0.2,setosa
2,4.9,3.0,1.4,0.2,setosa
3,4.7,3.2,1.3,0.2,setosa
4,4.6,3.1,1.5,0.2,setosa


## 步骤1：确定“科学”类型

In [19]:
schema(iris)

┌─────────────┬───────────────┬─────────────────────────────────┐
│ names       │ scitypes      │ types                           │
├─────────────┼───────────────┼─────────────────────────────────┤
│ SepalLength │ Continuous    │ Float64                         │
│ SepalWidth  │ Continuous    │ Float64                         │
│ PetalLength │ Continuous    │ Float64                         │
│ PetalWidth  │ Continuous    │ Float64                         │
│ Species     │ Multiclass{3} │ CategoricalValue{String, UInt8} │
└─────────────┴───────────────┴─────────────────────────────────┘


## 步骤2：划分数据

以下是我们如何将数据分割为目标和输入特征，这是MLJ监督模型所需的。我们同时随机化数据：

In [20]:
y, X = unpack(iris, ==(:class), name->true; rng=123);
scitype(y)

Table{Union{}}

这里有一种访问文档的方法

In [21]:
@doc unpack

```
unpack(table, f1, f2, ... fk;
       wrap_singles=false,
       shuffle=false,
       rng::Union{AbstractRNG,Int,Nothing}=nothing,
       coerce_options...)
```

Horizontally split any Tables.jl compatible `table` into smaller tables or vectors by making column selections determined by the predicates `f1`, `f2`, ..., `fk`. Selection from the column names is without replacement. A *predicate* is any object `f` such that `f(name)` is `true` or `false` for each column `name::Symbol` of `table`.

Returns a tuple of tables/vectors with length one greater than the number of supplied predicates, with the last component including all previously unselected columns.

```
julia> table = DataFrame(x=[1,2], y=['a', 'b'], z=[10.0, 20.0], w=["A", "B"])
2×4 DataFrame
 Row │ x      y     z        w
     │ Int64  Char  Float64  String
─────┼──────────────────────────────
   1 │     1  a        10.0  A
   2 │     2  b        20.0  B

Z, XY, W = unpack(table, ==(:z), !=(:w))
julia> Z
2-element Vector{Float64}:
 10.0
 20.0

julia> XY
2×2 DataFrame
 Row │ x      y
     │ Int64  Char
─────┼─────────────
   1 │     1  a
   2 │     2  b

julia> W  # the column(s) left over
2-element Vector{String}:
 "A"
 "B"
```

Whenever a returned table contains a single column, it is converted to a vector unless `wrap_singles=true`.

If `coerce_options` are specified then `table` is first replaced with `coerce(table, coerce_options)`. See [`ScientificTypes.coerce`](@ref) for details.

If `shuffle=true` then the rows of `table` are first shuffled, using the global RNG, unless `rng` is specified; if `rng` is an integer, it specifies the seed of an automatically generated Mersenne twister. If `rng` is specified then `shuffle=true` is implicit.


## 如何寻找模型？

以下是如何查看所有模型

In [23]:
all_models = models()

187-element Vector{NamedTuple{(:name, :package_name, :is_supervised, :abstract_type, :deep_properties, :docstring, :fit_data_scitype, :human_name, :hyperparameter_ranges, :hyperparameter_types, :hyperparameters, :implemented_methods, :inverse_transform_scitype, :is_pure_julia, :is_wrapper, :iteration_parameter, :load_path, :package_license, :package_url, :package_uuid, :predict_scitype, :prediction_type, :supports_class_weights, :supports_online, :supports_training_losses, :supports_weights, :transform_scitype, :input_scitype, :target_scitype, :output_scitype)}}:
 (name = ABODDetector, package_name = OutlierDetectionNeighbors, ... )
 (name = ABODDetector, package_name = OutlierDetectionPython, ... )
 (name = AEDetector, package_name = OutlierDetectionNetworks, ... )
 (name = ARDRegressor, package_name = ScikitLearn, ... )
 (name = AdaBoostClassifier, package_name = ScikitLearn, ... )
 (name = AdaBoostRegressor, package_name = ScikitLearn, ... )
 (name = AdaBoostStumpClassifier, package

每个条目都包含尚未加载定义代码的模型的元数据：

In [24]:
meta = all_models[3]

(name = "AEDetector",
 package_name = "OutlierDetectionNetworks",
 is_supervised = false,
 abstract_type = UnsupervisedDetector,
 deep_properties = (),
 docstring = "```\nAEDetector(encoder= Chain(),\n           decode...",
 fit_data_scitype =
     Union{Tuple{Union{Table{<:AbstractVector{<:Continuous}}, AbstractMatrix{<:Continuous}}}, Tuple{Union{Table{<:AbstractVector{<:Continuous}}, AbstractMatrix{<:Continuous}}, AbstractVector{<:OrderedFactor{2}}}},
 human_name = "ae detector",
 hyperparameter_ranges =
     (nothing, nothing, nothing, nothing, nothing, nothing, nothing, nothing),
 hyperparameter_types = ("Flux.Chain",
                         "Flux.Chain",
                         "Integer",
                         "Integer",
                         "Bool",
                         "Bool",
                         "Any",
                         "Function"),
 hyperparameters =
     (:encoder, :decoder, :batchsize, :epochs, :shuffle, :partial, :opt, :loss),
 implemented_methods = 

In [25]:
targetscitype = meta.target_scitype

AbstractVector{<:OrderedFactor{2}} (alias for AbstractArray{<:OrderedFactor{2}, 1})

In [26]:
scitype(y) <: targetscitype

false

让我们找到所有julia分类器：

In [27]:
filter_julia_classifiers(meta) =
    AbstractVector{Finite} <: meta.target_scitype &&
    meta.is_pure_julia

models(filter_julia_classifiers)

22-element Vector{NamedTuple{(:name, :package_name, :is_supervised, :abstract_type, :deep_properties, :docstring, :fit_data_scitype, :human_name, :hyperparameter_ranges, :hyperparameter_types, :hyperparameters, :implemented_methods, :inverse_transform_scitype, :is_pure_julia, :is_wrapper, :iteration_parameter, :load_path, :package_license, :package_url, :package_uuid, :predict_scitype, :prediction_type, :supports_class_weights, :supports_online, :supports_training_losses, :supports_weights, :transform_scitype, :input_scitype, :target_scitype, :output_scitype)}}:
 (name = AdaBoostStumpClassifier, package_name = DecisionTree, ... )
 (name = BayesianLDA, package_name = MultivariateStats, ... )
 (name = BayesianSubspaceLDA, package_name = MultivariateStats, ... )
 (name = ConstantClassifier, package_name = MLJModels, ... )
 (name = DecisionTreeClassifier, package_name = BetaML, ... )
 (name = DecisionTreeClassifier, package_name = DecisionTree, ... )
 (name = DeterministicConstantClassifie

查找名称中包含“分类器”的所有模型：

In [28]:
models("Classifier")

46-element Vector{NamedTuple{(:name, :package_name, :is_supervised, :abstract_type, :deep_properties, :docstring, :fit_data_scitype, :human_name, :hyperparameter_ranges, :hyperparameter_types, :hyperparameters, :implemented_methods, :inverse_transform_scitype, :is_pure_julia, :is_wrapper, :iteration_parameter, :load_path, :package_license, :package_url, :package_uuid, :predict_scitype, :prediction_type, :supports_class_weights, :supports_online, :supports_training_losses, :supports_weights, :transform_scitype, :input_scitype, :target_scitype, :output_scitype)}}:
 (name = AdaBoostClassifier, package_name = ScikitLearn, ... )
 (name = AdaBoostStumpClassifier, package_name = DecisionTree, ... )
 (name = BaggingClassifier, package_name = ScikitLearn, ... )
 (name = BernoulliNBClassifier, package_name = ScikitLearn, ... )
 (name = ComplementNBClassifier, package_name = ScikitLearn, ... )
 (name = ConstantClassifier, package_name = MLJModels, ... )
 (name = DecisionTreeClassifier, package_na

查找与数据匹配的所有（监督的）模型！

In [29]:
models(matching(X, y))

NamedTuple{(:name, :package_name, :is_supervised, :abstract_type, :deep_properties, :docstring, :fit_data_scitype, :human_name, :hyperparameter_ranges, :hyperparameter_types, :hyperparameters, :implemented_methods, :inverse_transform_scitype, :is_pure_julia, :is_wrapper, :iteration_parameter, :load_path, :package_license, :package_url, :package_uuid, :predict_scitype, :prediction_type, :supports_class_weights, :supports_online, :supports_training_losses, :supports_weights, :transform_scitype, :input_scitype, :target_scitype, :output_scitype)}[]

## 步骤3：选择并实例化模型

要加载定义新模型类型的代码，我们使用@load宏：

In [30]:
NeuralNetworkClassifier = @load NeuralNetworkClassifier

import MLJFlux ✔


┌ Info: For silent loading, specify `verbosity=0`. 
└ @ Main D:\TongYuan\.julia\packages\MLJModels\lDzCR\src\loading.jl:168


MLJFlux.NeuralNetworkClassifier

我们将使用超参数的默认值实例化此模型：

In [31]:
model = NeuralNetworkClassifier()

NeuralNetworkClassifier(
    builder = Short(
            n_hidden = 0,
            dropout = 0.5,
            σ = NNlib.σ),
    finaliser = NNlib.softmax,
    optimiser = Flux.Optimise.ADAM(0.001, (0.9, 0.999), 1.0e-8, IdDict{Any, Any}()),
    loss = Flux.Losses.crossentropy,
    epochs = 10,
    batch_size = 1,
    lambda = 0.0,
    alpha = 0.0,
    rng = Random._GLOBAL_RNG(),
    optimiser_changes_trigger_retraining = false,
    acceleration = CPU1{Nothing}(nothing))

In [32]:
info(model)

(name = "NeuralNetworkClassifier",
 package_name = "MLJFlux",
 is_supervised = true,
 abstract_type = Probabilistic,
 deep_properties = (:optimiser, :builder),
 docstring = "A neural network model for making probabilistic pr...",
 fit_data_scitype =
     Tuple{Table{<:AbstractVector{<:Continuous}}, AbstractVector{<:Finite}},
 human_name = "neural network classifier",
 hyperparameter_ranges = (nothing,
                          nothing,
                          nothing,
                          nothing,
                          nothing,
                          nothing,
                          nothing,
                          nothing,
                          nothing,
                          nothing,
                          nothing),
 hyperparameter_types = ("MLJFlux.Short",
                         "typeof(NNlib.softmax)",
                         "Flux.Optimise.ADAM",
                         "typeof(Flux.Losses.crossentropy)",
                         "Int64",
          

在MLJ中，模型只是一个包含超参数的结构，仅此而已。模型不存储已学习的参数。模型是可变的：

In [33]:
model.epochs = 12

12

所有模型都有一个关键字构造器，该构造器在执行@加载后即可工作：

In [34]:
NeuralNetworkClassifier(epochs=12) == model

true

## 模型训练、预测和测试

在MLJ中，模型和训练/验证数据通常在一台机器中：

In [37]:
mach = machine(model, X, y)

┌ Warning: The scitype of `X`, in `machine(model, X, ...)` is incompatible with `model=NeuralNetworkClassifier{Short,…}`:
│ scitype(X) = Table{Union{AbstractVector{Continuous}, AbstractVector{Multiclass{3}}}}
│ input_scitype(model) = Table{<:AbstractVector{<:Continuous}}.
└ @ MLJBase D:\TongYuan\.julia\packages\MLJBase\CMT6L\src\machines.jl:133
┌ Warning: The scitype of `y`, in `machine(model, X, y, ...)` is incompatible with `model=NeuralNetworkClassifier{Short,…}`:
│ scitype(y) = Table{Union{}}
│ target_scitype(model) = AbstractVector{<:Finite}.
└ @ MLJBase D:\TongYuan\.julia\packages\MLJBase\CMT6L\src\machines.jl:140


LoadError: DimensionMismatch("Differing number of observations in input and target. ")

除其他外，机器存储学习的参数。我们将用70%的数据对这台机器进行训练，并在30%的预留集上进行评估。让我们首先将所有行索引划分为列和测试子集：

In [36]:
train, test = partition(eachindex(y), 0.7)

LoadError: MethodError: no method matching keys(::DataFrame)
[0mClosest candidates are:
[0m  keys([91m::Union{Tables.AbstractColumns, Tables.AbstractRow}[39m) at D:\TongYuan\.julia\packages\Tables\PxO1m\src\Tables.jl:184
[0m  keys([91m::DataStructures.Accumulator[39m) at D:\TongYuan\.julia\packages\DataStructures\59MD0\src\accumulator.jl:58
[0m  keys([91m::DataFrames.GroupKey[39m) at D:\TongYuan\.julia\packages\DataFrames\zqFGs\src\groupeddataframe\groupeddataframe.jl:526
[0m  ...

In [44]:
MLJBase.fit!(mach, rows=train, verbosity=2)

LoadError: UndefVarError: train not defined

In [45]:
yhat = MLJBase.predict(mach, rows=test);  
yhat[1:3]

LoadError: UndefVarError: test not defined

训练完成后，可以检查学习到的参数：

In [46]:
fitted_params(mach)

LoadError: UndefVarError: mach not defined

In [47]:
report(mach) #获得训练细节

LoadError: UndefVarError: mach not defined

In [48]:
MLJ.save("neural_net.jlso", mach) #保存模型

LoadError: UndefVarError: mach not defined

In [49]:
mach2 = machine("neural_net.jlso")
yhat = MLJBase.predict(mach2, X);
yhat[1:3]

[warn | JLSO]: MethodError: no method matching deserialize(::Serialization.Serializer{IOBuffer}, ::Type{Flux.Dropout{Float64, Colon}})
Closest candidates are:
  deserialize(::Serialization.AbstractSerializer) at C:\Program Files\MWorks.Syslab 2022\Tools\julia-1.7.1\share\julia\stdlib\v1.7\Serialization\src\Serialization.jl:800
  deserialize(::Serialization.AbstractSerializer, !Matched::Type{T}) where T<:Distributed.WorkerPool at C:\Program Files\MWorks.Syslab 2022\Tools\julia-1.7.1\share\julia\stdlib\v1.7\Distributed\src\workerpool.jl:66
  deserialize(::Serialization.AbstractSerializer, !Matched::Type{T}) where T<:GPUArrays.AbstractGPUArray at D:\TongYuan\.julia\packages\GPUArrays\Zecv7\src\host\abstractarray.jl:37
  ...
[warn | JLSO]: EOFError: read end of file


LoadError: MethodError: no method matching Machine(::Vector{UInt8}; cache=true)
[0mClosest candidates are:
[0m  Machine([91m::M[39m, [91m::AbstractNode...[39m; cache) where M<:Model at D:\TongYuan\.julia\packages\MLJBase\CMT6L\src\machines.jl:44

如果要拟合检索到的模型，则需要将一些数据绑定到它：

In [50]:
mach3 = machine("neural_net.jlso", X, y)
MLJBase.fit!(mach3)

LoadError: UndefVarError: X not defined

机器会记住拟合期间使用的最后一组超参数，在迭代模型的情况下，它允许在仅增加迭代参数的情况下重新启动计算：

In [51]:
model.epochs = model.epochs + 4
MLJBase.fit!(mach, rows=train, verbosity=2)

LoadError: UndefVarError: train not defined

In [52]:
model.epochs = model.epochs + 4
model.optimiser.eta = 10*model.optimiser.eta
MLJBase.fit!(mach, rows=train, verbosity=2)

LoadError: UndefVarError: train not defined

但是，更改任何其他参数，训练将从头开始：

In [53]:
model.lambda = 0.001
MLJBase.fit!(mach, rows=train, verbosity=2)

LoadError: UndefVarError: train not defined

让我们静默训练总共 50 个 epoch，然后看一个预测：

In [54]:
model.epochs = 50
MLJBase.fit!(mach, rows=train)
yhat = MLJBase.predict(mach, X[test,:]); # or predict(mach, rows=test)
yhat[1]

LoadError: UndefVarError: train not defined

In [55]:
info(model).prediction_type

:probabilistic

对于大多数概率预测器，预测对象是 Distributions.Distribution 对象，支持此类对象的 Distributions.jl API。特别是，rand、pdf、logpdf、众数、中位数和均值方法将在适当的情况下适用。

所以，为了在第一次测试预测中获得“Iris-virginica”的概率:

In [56]:
pdf(yhat[1], "Iris-virginica")

LoadError: UndefVarError: yhat not defined

为了得到最有可能的观察结果:

In [57]:
mode(yhat[1])

LoadError: UndefVarError: yhat not defined

In [58]:
broadcast(pdf, yhat[1:4], "Iris-versicolor") #广播

LoadError: UndefVarError: yhat not defined

In [59]:
mode.(yhat[1:4])

LoadError: UndefVarError: yhat not defined

或者，您也可以使用 predict_mode 操作而不是 predict：

In [60]:
predict_mode(mach, X[test,:])[1:4] # or predict_mode(mach, rows=test)[1:4]

LoadError: UndefVarError: X not defined

In [61]:
L = levels(y)
pdf(yhat, L)[1:4, :]

LoadError: UndefVarError: y not defined

In [62]:
cross_entropy(yhat, y[test]) |> mean

LoadError: UndefVarError: y not defined

为了应用确定性度量，我们首先需要获得点估计：

In [63]:
misclassification_rate(mode.(yhat), y[test])

LoadError: UndefVarError: yhat not defined

我们顺便注意到，还有一个类似于模型的度量搜索工具：

In [64]:
measures()

63-element Vector{NamedTuple{(:name, :instances, :human_name, :target_scitype, :supports_weights, :supports_class_weights, :prediction_type, :orientation, :reports_each_observation, :aggregation, :is_feature_dependent, :docstring, :distribution_type)}}:
 (name = BrierLoss, instances = [brier_loss], ...)
 (name = BrierScore, instances = [brier_score], ...)
 (name = LPLoss, instances = [l1, l2], ...)
 (name = LogCoshLoss, instances = [log_cosh, log_cosh_loss], ...)
 (name = LogLoss, instances = [log_loss, cross_entropy], ...)
 (name = LogScore, instances = [log_score], ...)
 (name = SphericalScore, instances = [spherical_score], ...)
 (name = Accuracy, instances = [accuracy], ...)
 (name = AreaUnderCurve, instances = [area_under_curve, auc], ...)
 (name = BalancedAccuracy, instances = [balanced_accuracy, bacc, bac], ...)
 (name = ConfusionMatrix, instances = [confusion_matrix, confmat], ...)
 (name = FScore, instances = [f1score], ...)
 (name = FalseDiscoveryRate, instances = [false_disc

## 步骤4：评估模型性能

In [65]:
evaluate!(mach, resampling=Holdout(fraction_train=0.7),
          measures=[cross_entropy, brier_score])

LoadError: UndefVarError: mach not defined

或者改为应用交叉验证：

In [66]:
evaluate!(mach, resampling=CV(nfolds=6),
          measures=[cross_entropy, brier_score])

LoadError: UndefVarError: mach not defined

或者，蒙特卡洛交叉验证（交叉验证重复随机折叠）

In [67]:
e = evaluate!(mach, resampling=CV(nfolds=6, rng=123),
              repeats=3,
              measures=[cross_entropy, brier_score])

LoadError: UndefVarError: mach not defined

可以访问评估输出 e 的以下属性：measure、measurement、per_fold（每次折叠的测量）和 per_observation（每次观察的测量，如果报告）。

我们最后注意到，您可以通过指定 rows=.... 来限制从中绘制训练和测试折叠的观察行数。

In [68]:
train, test = partition(eachindex(y), 0.7)
mach = machine(model, X, y)
evaluate!(mach, resampling=CV(nfolds=6),
          measures=[cross_entropy, brier_score],
          rows=train)     # cv estimate, resampling from `train`
MLJBase.fit!(mach, rows=train)    # re-train using all of `train` observations
MLJBase.predict(mach, rows=test); # and predict missing targets

LoadError: UndefVarError: y not defined

## 学习曲线

由于我们的模型是一个迭代模型，我们可能希望将样本外性能作为迭代参数的函数进行检查。为此，我们可以使用 learning_curve 函数（顺便说一下，它可以应用于任何模型超参数）。首先为参数定义一个一维范围对象（在第 4 部分讨论调优时会详细介绍）：

In [69]:
r = range(model, :epochs, lower=1, upper=50, scale=:log)

NumericRange(1 ≤ epochs ≤ 50; origin=25.5, unit=24.5) on log scale

In [70]:
curve = learning_curve(mach,
                       range=r,
                       resampling=Holdout(fraction_train=0.7), # (default)
                       measure=cross_entropy)

using Plots
gr(size=(490,300))
plt=plot(curve.parameter_values, curve.measurements)
xlabel!(plt, "epochs")
ylabel!(plt, "cross entropy on holdout set")
savefig("learning_curve.png")
plt

LoadError: UndefVarError: mach not defined

当我们在第 4 部分查看调优时，我们将回到学习曲线。

## 第2部分资源

- From the MLJ manual:
    - [Getting Started](https://alan-turing-institute.github.io/MLJ.jl/dev/getting_started/)
    - [Model Search](https://alan-turing-institute.github.io/MLJ.jl/dev/model_search/)
    - [Evaluating Performance](https://alan-turing-institute.github.io/MLJ.jl/dev/evaluating_model_performance/) (using `evaluate!`)
    - [Learning Curves](https://alan-turing-institute.github.io/MLJ.jl/dev/learning_curves/)
    - [Performance Measures](https://alan-turing-institute.github.io/MLJ.jl/dev/performance_measures/) (loss functions, scores, etc)
- From Data Science Tutorials:
    - [Choosing and evaluating a model](https://juliaai.github.io/DataScienceTutorials.jl/getting-started/choosing-a-model/)
    - [Fit, predict, transform](https://juliaai.github.io/DataScienceTutorials.jl/getting-started/fit-and-predict/)

# 特征变换器（Transformers）与管道（Pipelines）

## 特征变换器

在训练期间不接收目标 y 的无监督模型总是有一个变换操作。它们有时还支持 inverse_transform 操作，意义明显，有时还支持预测操作（参见此处讨论的聚类示例）。否则，它们的处理方式很像监督模型。

这是一个简单的标准化示例：

In [71]:
x = rand(100);
@show mean(x) std(x);

mean(x) = 0.4704564851930727
std(x) = 0.29605733090559105


In [72]:
model = Standardizer() # a built-in model
mach = machine(model, x)
MLJBase.fit!(mach)
xhat = MLJBase.transform(mach, x);
@show mean(xhat) std(xhat);

mean(xhat) = 1.1879386363489176e-16
std(xhat) = 1.0


┌ Info: Training Machine{Standardizer,…}.
└ @ MLJBase D:\TongYuan\.julia\packages\MLJBase\CMT6L\src\machines.jl:464


这个特定的模型有一个 inverse_transform：

In [73]:
MLJBase.inverse_transform(mach, xhat) ≈ x

true

## 数据编码

让我们将所有 King County House 输入特征（参见示例 3）重新编码为一组连续特征。我们使用 ContinuousEncoder 模型执行此操作，默认情况下，该模型将：

one-hot 编码所有多类特征  
将所有 OrderedFactor 特征强制为连续特征  
将所有 Count 特征强制为 Continuous 特征  
删除任何剩余的非连续特征

首先，我们重新加载数据

In [74]:
file = CSV.File("D:\\Jupyter notebook\\DataSet\\house.csv");
house = DataFrames.DataFrame(file);
coerce!(house, autotype(file));
coerce!(house, Count => Continuous, :zipcode => Multiclass);
schema(house)

┌───────────────┬───────────────────┬───────────────────────────────────┐
│ names         │ scitypes          │ types                             │
├───────────────┼───────────────────┼───────────────────────────────────┤
│ price         │ Continuous        │ Float64                           │
│ bedrooms      │ OrderedFactor{13} │ CategoricalValue{Int64, UInt32}   │
│ bathrooms     │ OrderedFactor{30} │ CategoricalValue{Float64, UInt32} │
│ sqft_living   │ Continuous        │ Float64                           │
│ sqft_lot      │ Continuous        │ Float64                           │
│ floors        │ OrderedFactor{6}  │ CategoricalValue{Float64, UInt32} │
│ waterfront    │ OrderedFactor{2}  │ CategoricalValue{Int64, UInt32}   │
│ view          │ OrderedFactor{5}  │ CategoricalValue{Int64, UInt32}   │
│ condition     │ OrderedFactor{5}  │ CategoricalValue{Int64, UInt32}   │
│ grade         │ OrderedFactor{12} │ CategoricalValue{Int64, UInt32}   │
│ sqft_above    │ Continuous        │ 

In [75]:
y, X = unpack(house, ==(:price), name -> true, rng=123);

实例化无监督模型（特征变换器）：

In [76]:
encoder = ContinuousEncoder() # a built-in model; no need to @load it

ContinuousEncoder(
    drop_last = false,
    one_hot_ordered_factors = false)

将模型绑定到数据并拟合！

In [77]:
mach = machine(encoder, X) |> MLJBase.fit!;

┌ Info: Training Machine{ContinuousEncoder,…}.
└ @ MLJBase D:\TongYuan\.julia\packages\MLJBase\CMT6L\src\machines.jl:464


转换并检查结果：

In [78]:
Xcont = MLJBase.transform(mach, X);
schema(Xcont)

┌────────────────┬────────────┬─────────┐
│ names          │ scitypes   │ types   │
├────────────────┼────────────┼─────────┤
│ bedrooms       │ Continuous │ Float64 │
│ bathrooms      │ Continuous │ Float64 │
│ sqft_living    │ Continuous │ Float64 │
│ sqft_lot       │ Continuous │ Float64 │
│ floors         │ Continuous │ Float64 │
│ waterfront     │ Continuous │ Float64 │
│ view           │ Continuous │ Float64 │
│ condition      │ Continuous │ Float64 │
│ grade          │ Continuous │ Float64 │
│ sqft_above     │ Continuous │ Float64 │
│ sqft_basement  │ Continuous │ Float64 │
│ yr_built       │ Continuous │ Float64 │
│ zipcode__98001 │ Continuous │ Float64 │
│ zipcode__98002 │ Continuous │ Float64 │
│ zipcode__98003 │ Continuous │ Float64 │
│ zipcode__98004 │ Continuous │ Float64 │
│ zipcode__98005 │ Continuous │ Float64 │
│ zipcode__98006 │ Continuous │ Float64 │
│ zipcode__98007 │ Continuous │ Float64 │
│ zipcode__98008 │ Continuous │ Float64 │
│ zipcode__98010 │ Continuous │ Fl

## 更多的特征变换器

以下是列出 MLJ 的所有无监督模型的方法：

In [79]:
models(m->!m.is_supervised)

60-element Vector{NamedTuple{(:name, :package_name, :is_supervised, :abstract_type, :deep_properties, :docstring, :fit_data_scitype, :human_name, :hyperparameter_ranges, :hyperparameter_types, :hyperparameters, :implemented_methods, :inverse_transform_scitype, :is_pure_julia, :is_wrapper, :iteration_parameter, :load_path, :package_license, :package_url, :package_uuid, :predict_scitype, :prediction_type, :supports_class_weights, :supports_online, :supports_training_losses, :supports_weights, :transform_scitype, :input_scitype, :target_scitype, :output_scitype)}}:
 (name = ABODDetector, package_name = OutlierDetectionNeighbors, ... )
 (name = ABODDetector, package_name = OutlierDetectionPython, ... )
 (name = AEDetector, package_name = OutlierDetectionNetworks, ... )
 (name = AffinityPropagation, package_name = ScikitLearn, ... )
 (name = AgglomerativeClustering, package_name = ScikitLearn, ... )
 (name = BM25Transformer, package_name = MLJText, ... )
 (name = Birch, package_name = Sciki

Some commonly used ones are built-in (do not require `@load`ing):

model type                  | does what?
----------------------------|----------------------------------------------
ContinuousEncoder | transform input table to a table of `Continuous` features (see above)
FeatureSelector | retain or dump selected features
FillImputer | impute missing values
OneHotEncoder | one-hot encoder `Multiclass` (and optionally `OrderedFactor`) features
Standardizer | standardize (whiten) a vector or all `Continuous` features of a table
UnivariateBoxCoxTransformer | apply a learned Box-Cox transformation to a vector
UnivariateDiscretizer | discretize a `Continuous` vector, and hence render its elscitypw `OrderedFactor`

除了“动态”转换器（从数据中学习一些东西并且必须拟合！）之外，用户可以将普通函数包装为转换器，这样的静态转换器可以依赖于参数，就像动态的一样。

## 管道

In [80]:
length(schema(Xcont).names)

87

假设我们还想减少数据的维度。可以做到这一点的模型是来自 MultivariateStats 的 PCA：

In [81]:
PCA = @load PCA
reducer = PCA()

import MLJMultivariateStatsInterface ✔


┌ Info: For silent loading, specify `verbosity=0`. 
└ @ Main D:\TongYuan\.julia\packages\MLJModels\lDzCR\src\loading.jl:168


PCA(
    maxoutdim = 0,
    method = :auto,
    pratio = 0.99,
    mean = nothing)

现在，我们可以简单地重复上面的工作流程，将新的转换应用到 Xcont，我们可以将编码和降维模型组合成一个模型，称为管道。虽然 MLJ 为以各种方式组合模型提供了强大的接口，但我们现在将坚持使用这些最简单的组合模型类。构造它们的最简单方法是使用 @pipeline 宏：

In [82]:
pipe = @pipeline encoder reducer

Pipeline307(
    continuous_encoder = ContinuousEncoder(
            drop_last = false,
            one_hot_ordered_factors = false),
    pca = PCA(
            maxoutdim = 0,
            method = :auto,
            pratio = 0.99,
            mean = nothing))

请注意，管道是自动生成类型的一个实例（称为 Pipeline<some digits>）。

新模型的行为与任何其他特征变换器一样：

In [83]:
mach = machine(pipe, X)
MLJBase.fit!(mach)
Xsmall = MLJBase.transform(mach, X)
schema(Xsmall)

┌ Info: Training Machine{Pipeline307,…}.
└ @ MLJBase D:\TongYuan\.julia\packages\MLJBase\CMT6L\src\machines.jl:464
┌ Info: Training Machine{ContinuousEncoder,…}.
└ @ MLJBase D:\TongYuan\.julia\packages\MLJBase\CMT6L\src\machines.jl:464
┌ Info: Training Machine{PCA,…}.
└ @ MLJBase D:\TongYuan\.julia\packages\MLJBase\CMT6L\src\machines.jl:464
┌ Error: Problem fitting the machine Machine{PCA,…}. 
└ @ MLJBase D:\TongYuan\.julia\packages\MLJBase\CMT6L\src\machines.jl:594
┌ Info: Running type checks... 
└ @ MLJBase D:\TongYuan\.julia\packages\MLJBase\CMT6L\src\machines.jl:600
┌ Info: Type checks okay. 
└ @ MLJBase D:\TongYuan\.julia\packages\MLJBase\CMT6L\src\machines.jl:603
┌ Error: Problem fitting Machine{PCA,…}
└ @ MLJBase D:\TongYuan\.julia\packages\MLJBase\CMT6L\src\machines.jl:684
┌ Error: Problem fitting the machine Machine{Pipeline307,…}. 
└ @ MLJBase D:\TongYuan\.julia\packages\MLJBase\CMT6L\src\machines.jl:594
┌ Info: Running type checks... 
└ @ MLJBase D:\TongYuan\.julia\packages\

LoadError: TaskFailedException

[91m    nested task error: [39mMethodError: no method matching size(::MultivariateStats.PCA{Float64}, ::Int64)
    [0mClosest candidates are:
    [0m  size([91m::Union{LinearAlgebra.QR, LinearAlgebra.QRCompactWY, LinearAlgebra.QRPivoted}[39m, ::Integer) at C:\Program Files\MWorks.Syslab 2022\Tools\julia-1.7.1\share\julia\stdlib\v1.7\LinearAlgebra\src\qr.jl:566
    [0m  size([91m::Union{LinearAlgebra.Cholesky, LinearAlgebra.CholeskyPivoted}[39m, ::Integer) at C:\Program Files\MWorks.Syslab 2022\Tools\julia-1.7.1\share\julia\stdlib\v1.7\LinearAlgebra\src\cholesky.jl:495
    [0m  size([91m::Union{LinearAlgebra.Hermitian{T, S}, LinearAlgebra.Symmetric{T, S}} where {T, S}[39m, ::Any) at C:\Program Files\MWorks.Syslab 2022\Tools\julia-1.7.1\share\julia\stdlib\v1.7\LinearAlgebra\src\symmetric.jl:201
    [0m  ...
    Stacktrace:
     [1] [0m[1mfit_only![22m[0m[1m([22m[90mmach[39m::[0mMachine[90m{MLJMultivariateStatsInterface.PCA, true}[39

想要将此预处理与岭回归结合起来吗？

In [84]:
RidgeRegressor = @load RidgeRegressor pkg=MLJLinearModels
rgs = RidgeRegressor()
pipe2 = @pipeline encoder reducer rgs

import MLJLinearModels ✔


┌ Info: For silent loading, specify `verbosity=0`. 
└ @ Main D:\TongYuan\.julia\packages\MLJModels\lDzCR\src\loading.jl:168


Pipeline315(
    continuous_encoder = ContinuousEncoder(
            drop_last = false,
            one_hot_ordered_factors = false),
    pca = PCA(
            maxoutdim = 0,
            method = :auto,
            pratio = 0.99,
            mean = nothing),
    ridge_regressor = RidgeRegressor(
            lambda = 1.0,
            fit_intercept = true,
            penalize_intercept = false,
            scale_penalty_with_samples = true,
            solver = nothing))

现在我们的管道是一个监督模型，而不是一个特征变换器，我们可以评估其性能：

In [85]:
mach = machine(pipe2, X, y)
evaluate!(mach, measure=mae, resampling=Holdout()) # CV(nfolds=6) is default

┌ Error: Problem fitting the machine Machine{PCA,…}. 
└ @ MLJBase D:\TongYuan\.julia\packages\MLJBase\CMT6L\src\machines.jl:594
┌ Info: Running type checks... 
└ @ MLJBase D:\TongYuan\.julia\packages\MLJBase\CMT6L\src\machines.jl:600
┌ Info: Type checks okay. 
└ @ MLJBase D:\TongYuan\.julia\packages\MLJBase\CMT6L\src\machines.jl:603
┌ Error: Problem fitting Machine{PCA,…}
└ @ MLJBase D:\TongYuan\.julia\packages\MLJBase\CMT6L\src\machines.jl:684
┌ Error: Problem fitting the machine Machine{Pipeline315,…}. 
└ @ MLJBase D:\TongYuan\.julia\packages\MLJBase\CMT6L\src\machines.jl:594
┌ Info: Running type checks... 
└ @ MLJBase D:\TongYuan\.julia\packages\MLJBase\CMT6L\src\machines.jl:600
┌ Info: Type checks okay. 
└ @ MLJBase D:\TongYuan\.julia\packages\MLJBase\CMT6L\src\machines.jl:603


LoadError: TaskFailedException

[91m    nested task error: [39mMethodError: no method matching size(::MultivariateStats.PCA{Float64}, ::Int64)
    [0mClosest candidates are:
    [0m  size([91m::Union{LinearAlgebra.QR, LinearAlgebra.QRCompactWY, LinearAlgebra.QRPivoted}[39m, ::Integer) at C:\Program Files\MWorks.Syslab 2022\Tools\julia-1.7.1\share\julia\stdlib\v1.7\LinearAlgebra\src\qr.jl:566
    [0m  size([91m::Union{LinearAlgebra.Cholesky, LinearAlgebra.CholeskyPivoted}[39m, ::Integer) at C:\Program Files\MWorks.Syslab 2022\Tools\julia-1.7.1\share\julia\stdlib\v1.7\LinearAlgebra\src\cholesky.jl:495
    [0m  size([91m::Union{LinearAlgebra.Hermitian{T, S}, LinearAlgebra.Symmetric{T, S}} where {T, S}[39m, ::Any) at C:\Program Files\MWorks.Syslab 2022\Tools\julia-1.7.1\share\julia\stdlib\v1.7\LinearAlgebra\src\symmetric.jl:201
    [0m  ...
    Stacktrace:
     [1] [0m[1mfit_only![22m[0m[1m([22m[90mmach[39m::[0mMachine[90m{MLJMultivariateStatsInterface.PCA, true}[39

## 训练混合模型

现在注意如果我们对所有数据进行训练，然后更改回归量超参数并重新训练会发生什么：

In [86]:
MLJBase.fit!(mach)

┌ Info: Training Machine{Pipeline315,…}.
└ @ MLJBase D:\TongYuan\.julia\packages\MLJBase\CMT6L\src\machines.jl:464
┌ Info: Training Machine{ContinuousEncoder,…}.
└ @ MLJBase D:\TongYuan\.julia\packages\MLJBase\CMT6L\src\machines.jl:464
┌ Info: Training Machine{PCA,…}.
└ @ MLJBase D:\TongYuan\.julia\packages\MLJBase\CMT6L\src\machines.jl:464
┌ Error: Problem fitting the machine Machine{PCA,…}. 
└ @ MLJBase D:\TongYuan\.julia\packages\MLJBase\CMT6L\src\machines.jl:594
┌ Info: Running type checks... 
└ @ MLJBase D:\TongYuan\.julia\packages\MLJBase\CMT6L\src\machines.jl:600
┌ Info: Type checks okay. 
└ @ MLJBase D:\TongYuan\.julia\packages\MLJBase\CMT6L\src\machines.jl:603
┌ Error: Problem fitting Machine{PCA,…}
└ @ MLJBase D:\TongYuan\.julia\packages\MLJBase\CMT6L\src\machines.jl:684
┌ Error: Problem fitting the machine Machine{Pipeline315,…}. 
└ @ MLJBase D:\TongYuan\.julia\packages\MLJBase\CMT6L\src\machines.jl:594
┌ Info: Running type checks... 
└ @ MLJBase D:\TongYuan\.julia\packages\

LoadError: TaskFailedException

[91m    nested task error: [39mMethodError: no method matching size(::MultivariateStats.PCA{Float64}, ::Int64)
    [0mClosest candidates are:
    [0m  size([91m::Union{LinearAlgebra.QR, LinearAlgebra.QRCompactWY, LinearAlgebra.QRPivoted}[39m, ::Integer) at C:\Program Files\MWorks.Syslab 2022\Tools\julia-1.7.1\share\julia\stdlib\v1.7\LinearAlgebra\src\qr.jl:566
    [0m  size([91m::Union{LinearAlgebra.Cholesky, LinearAlgebra.CholeskyPivoted}[39m, ::Integer) at C:\Program Files\MWorks.Syslab 2022\Tools\julia-1.7.1\share\julia\stdlib\v1.7\LinearAlgebra\src\cholesky.jl:495
    [0m  size([91m::Union{LinearAlgebra.Hermitian{T, S}, LinearAlgebra.Symmetric{T, S}} where {T, S}[39m, ::Any) at C:\Program Files\MWorks.Syslab 2022\Tools\julia-1.7.1\share\julia\stdlib\v1.7\LinearAlgebra\src\symmetric.jl:201
    [0m  ...
    Stacktrace:
     [1] [0m[1mfit_only![22m[0m[1m([22m[90mmach[39m::[0mMachine[90m{MLJMultivariateStatsInterface.PCA, true}[39

In [87]:
pipe2.ridge_regressor.lambda = 0.1
MLJBase.fit!(mach)

┌ Info: Training Machine{Pipeline315,…}.
└ @ MLJBase D:\TongYuan\.julia\packages\MLJBase\CMT6L\src\machines.jl:464
┌ Info: Training Machine{ContinuousEncoder,…}.
└ @ MLJBase D:\TongYuan\.julia\packages\MLJBase\CMT6L\src\machines.jl:464
┌ Info: Training Machine{PCA,…}.
└ @ MLJBase D:\TongYuan\.julia\packages\MLJBase\CMT6L\src\machines.jl:464
┌ Error: Problem fitting the machine Machine{PCA,…}. 
└ @ MLJBase D:\TongYuan\.julia\packages\MLJBase\CMT6L\src\machines.jl:594
┌ Info: Running type checks... 
└ @ MLJBase D:\TongYuan\.julia\packages\MLJBase\CMT6L\src\machines.jl:600
┌ Info: Type checks okay. 
└ @ MLJBase D:\TongYuan\.julia\packages\MLJBase\CMT6L\src\machines.jl:603
┌ Error: Problem fitting Machine{PCA,…}
└ @ MLJBase D:\TongYuan\.julia\packages\MLJBase\CMT6L\src\machines.jl:684
┌ Error: Problem fitting the machine Machine{Pipeline315,…}. 
└ @ MLJBase D:\TongYuan\.julia\packages\MLJBase\CMT6L\src\machines.jl:594
┌ Info: Running type checks... 
└ @ MLJBase D:\TongYuan\.julia\packages\

LoadError: TaskFailedException

[91m    nested task error: [39mMethodError: no method matching size(::MultivariateStats.PCA{Float64}, ::Int64)
    [0mClosest candidates are:
    [0m  size([91m::Union{LinearAlgebra.QR, LinearAlgebra.QRCompactWY, LinearAlgebra.QRPivoted}[39m, ::Integer) at C:\Program Files\MWorks.Syslab 2022\Tools\julia-1.7.1\share\julia\stdlib\v1.7\LinearAlgebra\src\qr.jl:566
    [0m  size([91m::Union{LinearAlgebra.Cholesky, LinearAlgebra.CholeskyPivoted}[39m, ::Integer) at C:\Program Files\MWorks.Syslab 2022\Tools\julia-1.7.1\share\julia\stdlib\v1.7\LinearAlgebra\src\cholesky.jl:495
    [0m  size([91m::Union{LinearAlgebra.Hermitian{T, S}, LinearAlgebra.Symmetric{T, S}} where {T, S}[39m, ::Any) at C:\Program Files\MWorks.Syslab 2022\Tools\julia-1.7.1\share\julia\stdlib\v1.7\LinearAlgebra\src\symmetric.jl:201
    [0m  ...
    Stacktrace:
     [1] [0m[1mfit_only![22m[0m[1m([22m[90mmach[39m::[0mMachine[90m{MLJMultivariateStatsInterface.PCA, true}[39

第二次只重新训练岭回归器！

改变 PCA 模型和除 ContinuousEncoder 之外的每个模型的超参数（在重新训练之前出现）：

In [88]:
pipe2.pca.pratio = 0.9999
MLJBase.fit!(mach)

┌ Info: Training Machine{Pipeline315,…}.
└ @ MLJBase D:\TongYuan\.julia\packages\MLJBase\CMT6L\src\machines.jl:464
┌ Info: Training Machine{ContinuousEncoder,…}.
└ @ MLJBase D:\TongYuan\.julia\packages\MLJBase\CMT6L\src\machines.jl:464
┌ Info: Training Machine{PCA,…}.
└ @ MLJBase D:\TongYuan\.julia\packages\MLJBase\CMT6L\src\machines.jl:464
┌ Error: Problem fitting the machine Machine{PCA,…}. 
└ @ MLJBase D:\TongYuan\.julia\packages\MLJBase\CMT6L\src\machines.jl:594
┌ Info: Running type checks... 
└ @ MLJBase D:\TongYuan\.julia\packages\MLJBase\CMT6L\src\machines.jl:600
┌ Info: Type checks okay. 
└ @ MLJBase D:\TongYuan\.julia\packages\MLJBase\CMT6L\src\machines.jl:603
┌ Error: Problem fitting Machine{PCA,…}
└ @ MLJBase D:\TongYuan\.julia\packages\MLJBase\CMT6L\src\machines.jl:684
┌ Error: Problem fitting the machine Machine{Pipeline315,…}. 
└ @ MLJBase D:\TongYuan\.julia\packages\MLJBase\CMT6L\src\machines.jl:594
┌ Info: Running type checks... 
└ @ MLJBase D:\TongYuan\.julia\packages\

LoadError: TaskFailedException

[91m    nested task error: [39mMethodError: no method matching size(::MultivariateStats.PCA{Float64}, ::Int64)
    [0mClosest candidates are:
    [0m  size([91m::Union{LinearAlgebra.QR, LinearAlgebra.QRCompactWY, LinearAlgebra.QRPivoted}[39m, ::Integer) at C:\Program Files\MWorks.Syslab 2022\Tools\julia-1.7.1\share\julia\stdlib\v1.7\LinearAlgebra\src\qr.jl:566
    [0m  size([91m::Union{LinearAlgebra.Cholesky, LinearAlgebra.CholeskyPivoted}[39m, ::Integer) at C:\Program Files\MWorks.Syslab 2022\Tools\julia-1.7.1\share\julia\stdlib\v1.7\LinearAlgebra\src\cholesky.jl:495
    [0m  size([91m::Union{LinearAlgebra.Hermitian{T, S}, LinearAlgebra.Symmetric{T, S}} where {T, S}[39m, ::Any) at C:\Program Files\MWorks.Syslab 2022\Tools\julia-1.7.1\share\julia\stdlib\v1.7\LinearAlgebra\src\symmetric.jl:201
    [0m  ...
    Stacktrace:
     [1] [0m[1mfit_only![22m[0m[1m([22m[90mmach[39m::[0mMachine[90m{MLJMultivariateStatsInterface.PCA, true}[39

## 检查混合模型

上面用于更改嵌套超参数值的点语法在检查学习参数和训练复合模型时生成的报告时也很有用：

In [89]:
fitted_params(mach).ridge_regressor

LoadError: UndefRefError: access to undefined reference

In [90]:
report(mach).pca

LoadError: UndefRefError: access to undefined reference

## 合并目标特征变换器

接下来，假设我们不想使用原始的 :price 作为训练目标，而是使用对数价格（处理房价数据的常见做法）。然而，假设我们仍然想在原始线性尺度上报告最终预测（并将这些用于评估目的）。然后我们为关键字参数 target 和 inverse 提供适当的函数。

首先，我们将重载 log 和 exp 进行广播：

In [91]:
Base.log(v::AbstractArray) = log.(v)
Base.exp(v::AbstractArray) = exp.(v)

现在对于新管道：

In [92]:
pipe3 = @pipeline encoder reducer rgs target=log inverse=exp
mach = machine(pipe3, X, y)
evaluate!(mach, measure=mae)

┌ Info: Treating pipeline as a `Deterministic` predictor.
│ To override, specify `prediction_type=...` (options: :deterministic, :probabilistic, :interval). 
└ @ MLJBase D:\TongYuan\.julia\packages\MLJBase\CMT6L\src\composition\models\deprecated.jl:387
┌ Error: Problem fitting the machine Machine{PCA,…}. 
└ @ MLJBase D:\TongYuan\.julia\packages\MLJBase\CMT6L\src\machines.jl:594
┌ Info: Running type checks... 
└ @ MLJBase D:\TongYuan\.julia\packages\MLJBase\CMT6L\src\machines.jl:600
┌ Info: Type checks okay. 
└ @ MLJBase D:\TongYuan\.julia\packages\MLJBase\CMT6L\src\machines.jl:603
┌ Error: Problem fitting Machine{PCA,…}
└ @ MLJBase D:\TongYuan\.julia\packages\MLJBase\CMT6L\src\machines.jl:684
┌ Error: Problem fitting the machine Machine{Pipeline327,…}. 
└ @ MLJBase D:\TongYuan\.julia\packages\MLJBase\CMT6L\src\machines.jl:594
┌ Info: Running type checks... 
└ @ MLJBase D:\TongYuan\.julia\packages\MLJBase\CMT6L\src\machines.jl:600
┌ Info: Type checks okay. 
└ @ MLJBase D:\TongYuan\.juli

LoadError: TaskFailedException

[91m    nested task error: [39mMethodError: no method matching size(::MultivariateStats.PCA{Float64}, ::Int64)
    [0mClosest candidates are:
    [0m  size([91m::Union{LinearAlgebra.QR, LinearAlgebra.QRCompactWY, LinearAlgebra.QRPivoted}[39m, ::Integer) at C:\Program Files\MWorks.Syslab 2022\Tools\julia-1.7.1\share\julia\stdlib\v1.7\LinearAlgebra\src\qr.jl:566
    [0m  size([91m::Union{LinearAlgebra.Cholesky, LinearAlgebra.CholeskyPivoted}[39m, ::Integer) at C:\Program Files\MWorks.Syslab 2022\Tools\julia-1.7.1\share\julia\stdlib\v1.7\LinearAlgebra\src\cholesky.jl:495
    [0m  size([91m::Union{LinearAlgebra.Hermitian{T, S}, LinearAlgebra.Symmetric{T, S}} where {T, S}[39m, ::Any) at C:\Program Files\MWorks.Syslab 2022\Tools\julia-1.7.1\share\julia\stdlib\v1.7\LinearAlgebra\src\symmetric.jl:201
    [0m  ...
    Stacktrace:
     [1] [0m[1mfit_only![22m[0m[1m([22m[90mmach[39m::[0mMachine[90m{MLJMultivariateStatsInterface.PCA, true}[39

MLJ 还将允许您插入学习的目标转换。例如，我们可能希望将 Standardizer() 应用于目标，使其标准化，或 UnivariateBoxCoxTransformer() 使其看起来像高斯。然后，我们不指定目标函数，而是指定无监督模型（或模型类型）。一个没有指定 inverse 因为只允许实现 inverse_transform 的模型。

让我们看看这两个选项中的哪一个会产生更好的结果：

In [93]:
box = UnivariateBoxCoxTransformer(n=20)
stand = Standardizer()

pipe4 = @pipeline encoder reducer rgs target=box
mach = machine(pipe4, X, y)
evaluate!(mach, measure=mae)

┌ Info: Treating pipeline as a `Deterministic` predictor.
│ To override, specify `prediction_type=...` (options: :deterministic, :probabilistic, :interval). 
└ @ MLJBase D:\TongYuan\.julia\packages\MLJBase\CMT6L\src\composition\models\deprecated.jl:387
┌ Error: Problem fitting the machine Machine{PCA,…}. 
└ @ MLJBase D:\TongYuan\.julia\packages\MLJBase\CMT6L\src\machines.jl:594
┌ Info: Running type checks... 
└ @ MLJBase D:\TongYuan\.julia\packages\MLJBase\CMT6L\src\machines.jl:600
┌ Info: Type checks okay. 
└ @ MLJBase D:\TongYuan\.julia\packages\MLJBase\CMT6L\src\machines.jl:603
┌ Error: Problem fitting Machine{PCA,…}
└ @ MLJBase D:\TongYuan\.julia\packages\MLJBase\CMT6L\src\machines.jl:684
┌ Error: Problem fitting the machine Machine{Pipeline336,…}. 
└ @ MLJBase D:\TongYuan\.julia\packages\MLJBase\CMT6L\src\machines.jl:594
┌ Info: Running type checks... 
└ @ MLJBase D:\TongYuan\.julia\packages\MLJBase\CMT6L\src\machines.jl:600
┌ Info: Type checks okay. 
└ @ MLJBase D:\TongYuan\.juli

LoadError: TaskFailedException

[91m    nested task error: [39mMethodError: no method matching size(::MultivariateStats.PCA{Float64}, ::Int64)
    [0mClosest candidates are:
    [0m  size([91m::Union{LinearAlgebra.QR, LinearAlgebra.QRCompactWY, LinearAlgebra.QRPivoted}[39m, ::Integer) at C:\Program Files\MWorks.Syslab 2022\Tools\julia-1.7.1\share\julia\stdlib\v1.7\LinearAlgebra\src\qr.jl:566
    [0m  size([91m::Union{LinearAlgebra.Cholesky, LinearAlgebra.CholeskyPivoted}[39m, ::Integer) at C:\Program Files\MWorks.Syslab 2022\Tools\julia-1.7.1\share\julia\stdlib\v1.7\LinearAlgebra\src\cholesky.jl:495
    [0m  size([91m::Union{LinearAlgebra.Hermitian{T, S}, LinearAlgebra.Symmetric{T, S}} where {T, S}[39m, ::Any) at C:\Program Files\MWorks.Syslab 2022\Tools\julia-1.7.1\share\julia\stdlib\v1.7\LinearAlgebra\src\symmetric.jl:201
    [0m  ...
    Stacktrace:
     [1] [0m[1mfit_only![22m[0m[1m([22m[90mmach[39m::[0mMachine[90m{MLJMultivariateStatsInterface.PCA, true}[39

In [94]:
pipe4.target = stand
evaluate!(mach, measure=mae)

┌ Error: Problem fitting the machine Machine{PCA,…}. 
└ @ MLJBase D:\TongYuan\.julia\packages\MLJBase\CMT6L\src\machines.jl:594
┌ Info: Running type checks... 
└ @ MLJBase D:\TongYuan\.julia\packages\MLJBase\CMT6L\src\machines.jl:600
┌ Info: Type checks okay. 
└ @ MLJBase D:\TongYuan\.julia\packages\MLJBase\CMT6L\src\machines.jl:603
┌ Error: Problem fitting Machine{PCA,…}
└ @ MLJBase D:\TongYuan\.julia\packages\MLJBase\CMT6L\src\machines.jl:684
┌ Error: Problem fitting the machine Machine{Pipeline336,…}. 
└ @ MLJBase D:\TongYuan\.julia\packages\MLJBase\CMT6L\src\machines.jl:594
┌ Info: Running type checks... 
└ @ MLJBase D:\TongYuan\.julia\packages\MLJBase\CMT6L\src\machines.jl:600
┌ Info: Type checks okay. 
└ @ MLJBase D:\TongYuan\.julia\packages\MLJBase\CMT6L\src\machines.jl:603


LoadError: TaskFailedException

[91m    nested task error: [39mMethodError: no method matching size(::MultivariateStats.PCA{Float64}, ::Int64)
    [0mClosest candidates are:
    [0m  size([91m::Union{LinearAlgebra.QR, LinearAlgebra.QRCompactWY, LinearAlgebra.QRPivoted}[39m, ::Integer) at C:\Program Files\MWorks.Syslab 2022\Tools\julia-1.7.1\share\julia\stdlib\v1.7\LinearAlgebra\src\qr.jl:566
    [0m  size([91m::Union{LinearAlgebra.Cholesky, LinearAlgebra.CholeskyPivoted}[39m, ::Integer) at C:\Program Files\MWorks.Syslab 2022\Tools\julia-1.7.1\share\julia\stdlib\v1.7\LinearAlgebra\src\cholesky.jl:495
    [0m  size([91m::Union{LinearAlgebra.Hermitian{T, S}, LinearAlgebra.Symmetric{T, S}} where {T, S}[39m, ::Any) at C:\Program Files\MWorks.Syslab 2022\Tools\julia-1.7.1\share\julia\stdlib\v1.7\LinearAlgebra\src\symmetric.jl:201
    [0m  ...
    Stacktrace:
     [1] [0m[1mfit_only![22m[0m[1m([22m[90mmach[39m::[0mMachine[90m{MLJMultivariateStatsInterface.PCA, true}[39

## 第3部分资源

- From the MLJ manual:
    - [Transformers and other unsupervised models](https://alan-turing-institute.github.io/MLJ.jl/dev/transformers/)
    - [Linear pipelines](https://alan-turing-institute.github.io/MLJ.jl/dev/linear_pipelines/#Linear-Pipelines)
- From Data Science Tutorials:
    - [Composing models](https://juliaai.github.io/DataScienceTutorials.jl/getting-started/composing-models/)

# 调参

## 单一参数调整

调整单个超参数最简单的方法是使用 learning_curve，我们已经在第 2 部分中看到过。让我们在 Horse Colic 分类问题中看到这一点，在要调整的参数是嵌套的情况下（因为模型是管道）：

In [95]:
y, X = unpack(horse, ==(:outcome), name -> true);

LogisticClassifier = @load LogisticClassifier pkg=MLJLinearModels
model = @pipeline Standardizer ContinuousEncoder LogisticClassifier
mach = machine(model, X, y)

import MLJLinearModels ✔


┌ Info: For silent loading, specify `verbosity=0`. 
└ @ Main D:\TongYuan\.julia\packages\MLJModels\lDzCR\src\loading.jl:168


Machine{Pipeline344,…} trained 0 times; caches data
  model: Pipeline344
  args: 
    1:	Source @126 ⏎ `Table{Union{AbstractVector{Continuous}, AbstractVector{Multiclass{3}}, AbstractVector{Multiclass{2}}, AbstractVector{Multiclass{6}}, AbstractVector{OrderedFactor{4}}, AbstractVector{OrderedFactor{5}}, AbstractVector{OrderedFactor{2}}}}`
    2:	Source @351 ⏎ `AbstractVector{Multiclass{3}}`


In [96]:
r = range(model, :(logistic_classifier.lambda), lower = 1e-2, upper=100, scale=:log10)

NumericRange(0.01 ≤ logistic_classifier.lambda ≤ 100; origin=50.005, unit=49.995) on log10 scale

如果你很好奇，你可以看到这个范围对于给定的分辨率会生成什么 lambda 值：

In [97]:
iterator(r, 5)

5-element Vector{Float64}:
   0.01
   0.1
   1.0
  10.0
 100.0

In [98]:
_, _, lambdas, losses = learning_curve(mach,
                                       range=r,
                                       resampling=CV(nfolds=6),
                                       resolution=30, # default
                                       measure=cross_entropy)
plt=plot(lambdas, losses, xscale=:log10)
xlabel!(plt, "lambda")
ylabel!(plt, "cross entropy using 6-fold CV")
savefig("learning_curve2.png")
plt


best_lambda = lambdas[argmin(losses)]

┌ Info: Training Machine{ProbabilisticTunedModel{Grid,…},…}.
└ @ MLJBase D:\TongYuan\.julia\packages\MLJBase\CMT6L\src\machines.jl:464
┌ Info: Attempting to evaluate 30 models.
└ @ MLJTuning D:\TongYuan\.julia\packages\MLJTuning\Al9yX\src\tuned_models.jl:680
Evaluating over 30 metamodels: 100%[=========================] Time: 0:03:32


LoadError: UndefVarError: plot not defined

## 自调整模型

查看超参数调整（受 MLR 启发）的一种更复杂的方法是作为模型包装器。包装模型本身就是一个新模型，当您适合它时，它会在对所有提供的数据进行训练之前调整被包装模型的指定超参数。在包装模型上调用 predict 就像在原始模型上调用 predict 一样，但超参数已经优化。

换句话说，我们可以将包装模型视为原始模型的“自我调整”版本。

我们现在创建上面管道的自调整版本，将来自 ContinuousEncoder 的参数添加到我们想要优化的参数中。

首先，让我们选择一个调优策略（从这些选项中）。 MLJ 支持普通的 Grid 搜索（详情查询 ?Grid）。但是，由于网格搜索的实用性仅限于少量参数，并且网格搜索在其他地方进行了演示（请参阅下面的资源），我们将在此处演示 RandomSearch：

In [99]:
tuning = RandomSearch(rng=123)

RandomSearch(
    bounded = Distributions.Uniform,
    positive_unbounded = Distributions.Gamma,
    other = Distributions.Normal,
    rng = Random.MersenneTwister(123))

在此策略中，每个参数都根据预先指定的先验分布进行采样，该先验分布适合使用之前使用范围构造的一维范围对象。虽然人们对先验的规范有很多控制（运行 ?RandomSearch 了解详细信息），但我们会让算法自动生成这些先验。

### 无界范围和采样

在 MLJ 中，范围不必有界。在 RandomSearch 中，默认情况下，使用 Gamma 分布对正的无界范围进行采样：

In [100]:
r = range(model,
          :(logistic_classifier.lambda),
          lower=0,
          origin=6,
          unit=5,
          scale=:log10)

NumericRange(0 ≤ logistic_classifier.lambda ≤ Inf; origin=6.0, unit=5.0) on log10 scale

范围内的比例在 RandomSearch 中无效（除非它是一个函数），但这会影响后面的图，但它确实会影响后面的图。

让我们看看使用 Gamma 分布的采样对于这个范围意味着什么：

In [101]:
import Distributions
sampler_r = sampler(r, Distributions.Gamma)
plt = histogram(rand(sampler_r, 10000), nbins=50)
savefig("gamma_sampler.png")
plt

LoadError: UndefVarError: histogram not defined

我们将添加到其中的第二个参数是名义（有限），默认情况下，将统一采样。由于它是标称的，我们指定值而不是上限和下限：

In [102]:
s  = range(model, :(continuous_encoder.one_hot_ordered_factors),
           values = [true, false])

NominalRange(continuous_encoder.one_hot_ordered_factors = true, false)

### 调优器

In [103]:
现在对于优化器，它是 TunedModel 的一个实例：

LoadError: syntax: invalid character "，" near column 15

In [104]:
tuned_model = TunedModel(model=model,
                         ranges=[r, s],
                         resampling=CV(nfolds=6),
                         measures=cross_entropy,
                         tuning=tuning,
                         n=15)

ProbabilisticTunedModel(
    model = Pipeline344(
            standardizer = Standardizer,
            continuous_encoder = ContinuousEncoder,
            logistic_classifier = LogisticClassifier),
    tuning = RandomSearch(
            bounded = Distributions.Uniform,
            positive_unbounded = Distributions.Gamma,
            other = Distributions.Normal,
            rng = Random.MersenneTwister(123)),
    resampling = CV(
            nfolds = 6,
            shuffle = false,
            rng = Random._GLOBAL_RNG()),
    measure = LogLoss(tol = 2.220446049250313e-16),
    weights = nothing,
    operation = nothing,
    range = ParamRange[NumericRange(0 ≤ logistic_classifier.lambda ≤ Inf; origin=6.0, unit=5.0) on log10 scale, NominalRange(continuous_encoder.one_hot_ordered_factors = true, false)],
    selection_heuristic = NaiveSelection(nothing),
    train_best = true,
    repeats = 1,
    n = 15,
    acceleration = CPU1{Nothing}(nothing),
    acceleration_resampling = CPU1{Nothi

我们可以将 fit!/predict 工作流程应用于 tune_model ，就像对任何其他模型一样：

In [105]:
tuned_mach = machine(tuned_model, X, y);
MLJBase.fit!(tuned_mach);
MLJBase.predict(tuned_mach, rows=1:3)

┌ Info: Training Machine{ProbabilisticTunedModel{RandomSearch,…},…}.
└ @ MLJBase D:\TongYuan\.julia\packages\MLJBase\CMT6L\src\machines.jl:464
┌ Info: Attempting to evaluate 15 models.
└ @ MLJTuning D:\TongYuan\.julia\packages\MLJTuning\Al9yX\src\tuned_models.jl:680
Evaluating over 15 metamodels: 100%[=========================] Time: 0:01:22


3-element UnivariateFiniteVector{Multiclass{3}, Int64, UInt32, Float64}:
 UnivariateFinite{Multiclass{3}}(1=>0.519, 2=>0.319, 3=>0.161)
 UnivariateFinite{Multiclass{3}}(1=>0.555, 2=>0.241, 3=>0.203)
 UnivariateFinite{Multiclass{3}}(1=>0.737, 2=>0.169, 3=>0.0946)

可以从详细报告中检查调整的结果。例如，我们有：

In [106]:
rep = report(tuned_mach);
rep.best_model

Pipeline344(
    standardizer = Standardizer(
            features = Symbol[],
            ignore = false,
            ordered_factor = false,
            count = false),
    continuous_encoder = ContinuousEncoder(
            drop_last = false,
            one_hot_ordered_factors = true),
    logistic_classifier = LogisticClassifier(
            lambda = 0.9590656995775001,
            gamma = 0.0,
            penalty = :l2,
            fit_intercept = true,
            penalize_intercept = false,
            scale_penalty_with_samples = true,
            solver = nothing))

默认情况下，有界范围的采样是统一的。

在双参数的特殊情况下，您还可以绘制结果：

In [107]:
plt = plot(tuned_mach)
savefig("tuning.png")
plt

LoadError: UndefVarError: plot not defined

最后，让我们比较一下自调整模型与原始模型的性能的交叉验证估计（这里是嵌套重采样的示例）：

In [108]:
err = evaluate!(mach, resampling=CV(nfolds=3), measure=cross_entropy)

Evaluating over 3 folds: 100%[=========================] Time: 0:00:02


PerformanceEvaluation object with these fields:
  measure, measurement, operation, per_fold,
  per_observation, fitted_params_per_fold,
  report_per_fold, train_test_pairs
Extract:
┌────────────────────────────┬─────────────┬───────────┬────────────────────────
│ measure                    │ measurement │ operation │ per_fold              ⋯
├────────────────────────────┼─────────────┼───────────┼────────────────────────
│ LogLoss(tol = 2.22045e-16) │ 0.823       │ predict   │ [0.826, 0.828, 0.815] ⋯
└────────────────────────────┴─────────────┴───────────┴────────────────────────


In [109]:
tuned_err = evaluate!(tuned_mach, resampling=CV(nfolds=3), measure=cross_entropy)

Evaluating over 3 folds:  67%[================>        ]  ETA: 0:01:15┌ Error: Problem fitting the machine Machine{Pipeline344,…}. 
└ @ MLJBase D:\TongYuan\.julia\packages\MLJBase\CMT6L\src\machines.jl:594
┌ Info: Running type checks... 
└ @ MLJBase D:\TongYuan\.julia\packages\MLJBase\CMT6L\src\machines.jl:600
┌ Info: Type checks okay. 
└ @ MLJBase D:\TongYuan\.julia\packages\MLJBase\CMT6L\src\machines.jl:603
┌ Error: Problem fitting the machine Machine{ProbabilisticTunedModel{RandomSearch,…},…}. 
└ @ MLJBase D:\TongYuan\.julia\packages\MLJBase\CMT6L\src\machines.jl:594
┌ Info: Running type checks... 
└ @ MLJBase D:\TongYuan\.julia\packages\MLJBase\CMT6L\src\machines.jl:600
┌ Info: Type checks okay. 
└ @ MLJBase D:\TongYuan\.julia\packages\MLJBase\CMT6L\src\machines.jl:603


LoadError: InterruptException:

## 第4部分资源

- From the MLJ manual:
   - [Learning Curves](https://alan-turing-institute.github.io/MLJ.jl/dev/learning_curves/)
   - [Tuning Models](https://alan-turing-institute.github.io/MLJ.jl/dev/tuning_models/)
- The [MLJTuning repo](https://github.com/juliaai/MLJTuning.jl#who-is-this-repo-for) - mostly for developers

- From Data Science Tutorials:
    - [Tuning a model](https://juliaai.github.io/DataScienceTutorials.jl/getting-started/model-tuning/)
    - [Crabs with XGBoost](https://juliaai.github.io/DataScienceTutorials.jl/end-to-end/crabs-xgb/) `Grid` tuning in stages for a tree-boosting model with many parameters
    - [Boston with LightGBM](https://juliaai.github.io/DataScienceTutorials.jl/end-to-end/boston-lgbm/) -  `Grid` tuning for another popular tree-booster
    - [Boston with Flux](https://juliaai.github.io/DataScienceTutorials.jl/end-to-end/boston-flux/) - optimizing batch size in a simple neural network regressor
- [UCI Horse Colic Data Set](http://archive.ics.uci.edu/ml/datasets/Horse+Colic)

# 高级模型

虽然@pipeline 非常适合以非分支序列组合模型，但对于更复杂的模型组合，您需要使用 MLJ 的通用模型组合语法。主要有两个步骤：

通过构建学习网络对复合模型进行原型制作，在构建时可以在一些（虚拟）数据上对其进行测试。  
将学习网络导出为新的独立模型类型。  
与管道模型一样，导出模型类型的实例的行为类似于任何其他模型（并且不绑定到任何数据，直到您将它们包装在机器中）。  

## 使用组合构建管道

In [110]:
pipe = @pipeline Standardizer LogisticClassifier;

使用通用语法。

下面是一些我们将用来测试我们的学习网络的虚拟数据：

In [111]:
X, y = make_blobs(5, 3)
pretty(X)

┌────────────┬────────────┬────────────┐
│ x1         │ x2         │ x3         │
│ Float64    │ Float64    │ Float64    │
│ Continuous │ Continuous │ Continuous │
├────────────┼────────────┼────────────┤
│ -7.6775    │ -6.47767   │ 3.30184    │
│ 2.40296    │ -4.2438    │ 2.54009    │
│ -6.8739    │ -7.84651   │ -0.598379  │
│ -0.756992  │ -2.51176   │ 2.92716    │
│ -8.68655   │ -4.87497   │ 1.17124    │
└────────────┴────────────┴────────────┘


第 0 步 - 使用所有可用于训练、转换和预测的数据，就像“手动”组合模型一样进行：

In [112]:
stand = Standardizer();
linear = LogisticClassifier();

mach1 = machine(stand, X);
MLJBase.fit!(mach1);
Xstand = MLJBase.transform(mach1, X);

mach2 = machine(linear, Xstand, y);
MLJBase.fit!(mach2);
yhat = MLJBase.predict(mach2, Xstand)

┌ Info: Training Machine{Standardizer,…}.
└ @ MLJBase D:\TongYuan\.julia\packages\MLJBase\CMT6L\src\machines.jl:464
┌ Info: Training Machine{LogisticClassifier,…}.
└ @ MLJBase D:\TongYuan\.julia\packages\MLJBase\CMT6L\src\machines.jl:464
┌ Info: Solver: MLJLinearModels.LBFGS()
└ @ MLJLinearModels D:\TongYuan\.julia\packages\MLJLinearModels\eRCwo\src\mlj\interface.jl:76


5-element UnivariateFiniteVector{Multiclass{3}, Int64, UInt32, Float64}:
 UnivariateFinite{Multiclass{3}}(1=>0.153, 2=>0.157, 3=>0.69)
 UnivariateFinite{Multiclass{3}}(1=>0.309, 2=>0.283, 3=>0.408)
 UnivariateFinite{Multiclass{3}}(1=>0.11, 2=>0.0902, 3=>0.8)
 UnivariateFinite{Multiclass{3}}(1=>0.281, 2=>0.321, 3=>0.397)
 UnivariateFinite{Multiclass{3}}(1=>0.139, 2=>0.157, 3=>0.704)

第 1 步 - 编辑您的代码，如下所示：

In [113]:
X = source(X)  # or X = source() if not testing
y = source(y)  # or y = source()

stand = Standardizer();
linear = LogisticClassifier();

mach1 = machine(stand, X);
Xstand = MLJBase.transform(mach1, X);

mach2 = machine(linear, Xstand, y);
yhat = MLJBase.predict(mach2, Xstand)

Node{Machine{LogisticClassifier,…}}
  args:
    1:	Node{Machine{Standardizer,…}}
  formula:
    predict(
        Machine{LogisticClassifier,…}, 
        transform(
            Machine{Standardizer,…}, 
            Source @025))

现在 X、y、Xstand 和 yhat 是节点（“变量”或“动态数据”）而不是数据。现在，只要我们合适，所有的训练、预测和转换都是惰性执行的！这些节点之一。我们调用一个节点来检索它在原始手动工作流中表示的数据。

In [114]:
MLJBase.fit!(Xstand)
Xstand() |> pretty

┌────────────┬────────────┬────────────┐
│ x1         │ x2         │ x3         │
│ Float64    │ Float64    │ Float64    │
│ Continuous │ Continuous │ Continuous │
├────────────┼────────────┼────────────┤
│ -0.690163  │ -0.62632   │ 0.897753   │
│ 1.38097    │ 0.461022   │ 0.420679   │
│ -0.525055  │ -1.29261   │ -1.54491   │
│ 0.731728   │ 1.3041     │ 0.663093   │
│ -0.897484  │ 0.153802   │ -0.436616  │
└────────────┴────────────┴────────────┘


┌ Info: Training Machine{Standardizer,…}.
└ @ MLJBase D:\TongYuan\.julia\packages\MLJBase\CMT6L\src\machines.jl:464


In [115]:
MLJBase.fit!(yhat);
yhat()

┌ Info: Not retraining Machine{Standardizer,…}. Use `force=true` to force.
└ @ MLJBase D:\TongYuan\.julia\packages\MLJBase\CMT6L\src\machines.jl:467
┌ Info: Training Machine{LogisticClassifier,…}.
└ @ MLJBase D:\TongYuan\.julia\packages\MLJBase\CMT6L\src\machines.jl:464
┌ Info: Solver: MLJLinearModels.LBFGS()
└ @ MLJLinearModels D:\TongYuan\.julia\packages\MLJLinearModels\eRCwo\src\mlj\interface.jl:76


5-element UnivariateFiniteVector{Multiclass{3}, Int64, UInt32, Float64}:
 UnivariateFinite{Multiclass{3}}(1=>0.153, 2=>0.157, 3=>0.69)
 UnivariateFinite{Multiclass{3}}(1=>0.309, 2=>0.283, 3=>0.408)
 UnivariateFinite{Multiclass{3}}(1=>0.11, 2=>0.0902, 3=>0.8)
 UnivariateFinite{Multiclass{3}}(1=>0.281, 2=>0.321, 3=>0.397)
 UnivariateFinite{Multiclass{3}}(1=>0.139, 2=>0.157, 3=>0.704)

节点 yhat 是唯一源节点的“后代”（在我们定义的关联 DAG 中）：

In [116]:
sources(yhat)

2-element Vector{Any}:
 Source @025 ⏎ `Table{AbstractVector{Continuous}}`
 Source @337 ⏎ `AbstractVector{Multiclass{3}}`

当我们像这样调用 yhat 时，源节点的数据被 Xnew 替换以获得新的预测：

In [117]:
Xnew, _ = make_blobs(2, 3);
yhat(Xnew)

2-element UnivariateFiniteVector{Multiclass{3}, Int64, UInt32, Float64}:
 UnivariateFinite{Multiclass{3}}(1=>0.244, 2=>0.724, 3=>0.0315)
 UnivariateFinite{Multiclass{3}}(1=>0.383, 2=>0.335, 3=>0.282)

第 2 步 - 将学习网络导出为新的独立模型类型

现在，有点自相矛盾的是，我们可以在定义新模型类型之前将整个网络包装在一个特殊的机器中——称为学习网络机器。确实这样做是导出过程中的必要步骤，因为这台机器会告诉导出宏：

In [118]:
surrogate = Probabilistic()     # a model with no fields!
mach = machine(surrogate, X, y; predict=yhat)

Machine{ProbabilisticSurrogate,…} trained 0 times; does not cache data
  model: ProbabilisticSurrogate
  args: 
    1:	Source @025 ⏎ `Table{AbstractVector{Continuous}}`
    2:	Source @337 ⏎ `AbstractVector{Multiclass{3}}`


虽然我们并没有真正需要使用它，但这台机器的行为就像你期望的那样：

In [119]:
Xnew, _ = make_blobs(2, 3)
MLJBase.fit!(mach)
MLJBase.predict(mach, Xnew)

┌ Info: Not retraining Machine{Standardizer,…}. Use `force=true` to force.
└ @ MLJBase D:\TongYuan\.julia\packages\MLJBase\CMT6L\src\machines.jl:467
┌ Info: Not retraining Machine{LogisticClassifier,…}. Use `force=true` to force.
└ @ MLJBase D:\TongYuan\.julia\packages\MLJBase\CMT6L\src\machines.jl:467


2-element UnivariateFiniteVector{Multiclass{3}, Int64, UInt32, Float64}:
 UnivariateFinite{Multiclass{3}}(1=>0.242, 2=>0.67, 3=>0.0876)
 UnivariateFinite{Multiclass{3}}(1=>0.227, 2=>0.292, 3=>0.481)

现在我们使用适当修饰的 Julia 结构定义创建一个新的模型类型：

In [120]:
@from_network mach begin
    mutable struct YourPipe
        standardizer = stand
        classifier = linear::Probabilistic
    end
end

实例化和评估一些新数据：

In [121]:
pipe = YourPipe()
X, y = @load_iris;   # built-in data set
mach = machine(pipe, X, y)
evaluate!(mach, measure=misclassification_rate, operation=predict_mode)

Evaluating over 6 folds: 100%[=========================] Time: 0:00:07


PerformanceEvaluation object with these fields:
  measure, measurement, operation, per_fold,
  per_observation, fitted_params_per_fold,
  report_per_fold, train_test_pairs
Extract:
┌─────────────────────────┬─────────────┬──────────────┬────────────────────────
│ measure                 │ measurement │ operation    │ per_fold              ⋯
├─────────────────────────┼─────────────┼──────────────┼────────────────────────
│ MisclassificationRate() │ 0.427       │ predict_mode │ [0.0, 0.0, 0.96, 0.92 ⋯
└─────────────────────────┴─────────────┴──────────────┴────────────────────────
                                                                1 column omitted


## 两个回归预测因子平均值的复合模型

我们将定义一个复合模型：

标准化输入数据  
学习并将 Box-Cox 变换应用于目标变量  
混合了两个监督学习模型的预测——一个岭回归器和一个随机森林回归器；我们将使用简单的平均值进行混合（有关更复杂的堆叠示例，请参见此处）  
将逆 Box-Cox 变换应用于此混合预测  

In [122]:
RandomForestRegressor = @load RandomForestRegressor pkg=DecisionTree

import MLJDecisionTreeInterface ✔


┌ Info: For silent loading, specify `verbosity=0`. 
└ @ Main D:\TongYuan\.julia\packages\MLJModels\lDzCR\src\loading.jl:168


MLJDecisionTreeInterface.RandomForestRegressor

输入层

In [123]:
X = source()
y = source()

第一层和目标变换

In [ ]:
std_model = Standardizer()
stand = machine(std_model, X)
W = MLJ.transform(stand, X)

box_model = UnivariateBoxCoxTransformer()
box = machine(box_model, y)
z = MLJ.transform(box, y)

第二层

In [ ]:
ridge_model = RidgeRegressor(lambda=0.1)
ridge = machine(ridge_model, W, z)

forest_model = RandomForestRegressor()
forest = machine(forest_model, W, z)

ẑ = 0.5*MLJBase.predict(ridge, W) + 0.5*MLJBase.predict(forest, W)

输出层

In [ ]:
ŷ = MLJBase.inverse_transform(box, ẑ)

定义学习网络后，我们准备导出：

In [ ]:
@from_network machine(Deterministic(), X, y, predict=ŷ) begin
    mutable struct CompositeModel
        rgs1 = ridge_model
        rgs2 = forest_model
    end
end

让我们实例化新的模型类型并在一些数据上进行尝试：

In [ ]:
composite = CompositeModel()

In [ ]:
X, y = @load_boston;
mach = machine(composite, X, y);
evaluate!(mach,
          resampling=CV(nfolds=6, shuffle=true),
          measures=[rms, mae])

## 第5部分资源

- From the MLJ manual:
   - [Learning Networks](https://alan-turing-institute.github.io/MLJ.jl/stable/composing_models/#Learning-Networks-1)
- From Data Science Tutorials:
    - [Learning Networks](https://juliaai.github.io/DataScienceTutorials.jl/getting-started/learning-networks/)
    - [Learning Networks 2](https://juliaai.github.io/DataScienceTutorials.jl/getting-started/learning-networks-2/)